In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from joblib import dump, load
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge, Lasso
import tensorflow as tf
import warnings
from TransformationPipeline import TransformationPipeline
warnings.filterwarnings('ignore')

### **1. Loading Data**

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.head()

,ID,marka,naw3,9at3a,khochn,toul,3ordh,R,G,B,soum
0,0,Bey&Bey,mta3 a3res,kabbout,1.081667,22.4,57.0,98,18,21,49.2
1,1,HA,confy,sabbat,1.038333,46.4,54.0,128,23,27,180.9
2,2,Zen,confy,maryoul,1.021667,40.8,54.0,121,22,25,143.8
3,3,Bey&Bey,confy,maryoul,1.065000,24.8,56.0,101,18,21,54.4
4,4,Armani,classy,kabbout,1.041667,72.0,61.0,146,26,30,345.9


In [4]:
# df.rename(columns={'marka': 'Brand' ,"naw3" : "Class" , "9at3a" : "Category" , "toul" : "Height" , "khochn":"Depth", "3ordh": "Width","soum":"Price"}, inplace=True)
# df =df.drop(columns="ID")

In [5]:
data['9at3a'] = data['9at3a'].replace('kabbout', 'Jacket')
data['9at3a'] = data['9at3a'].replace('sabbat', 'Shoes')
data['9at3a'] = data['9at3a'].replace('maryoul', 'Shirt')
data['9at3a'] = data['9at3a'].replace('t-shirt', 'T_shirt')
data['9at3a'] = data['9at3a'].replace('jacket', 'Jacket')
data['9at3a'] = data['9at3a'].replace('serwel', 'Pants')
data = data[data['9at3a'] != 'hoodie']
data = data[data['9at3a'] != 'socks']
data =data.drop(columns="ID")

In [6]:
data["9at3a"].unique()

array(['Jacket', 'Shoes', 'Shirt', 'T_shirt', 'Pants'], dtype=object)

### **2. Data Checks**

In [7]:
data.isna().sum()

marka     0
naw3      0
9at3a     0
khochn    0
toul      0
3ordh     0
R         0
G         0
B         0
soum      0
dtype: int64

In [8]:
data.duplicated().sum()

366

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46290 entries, 0 to 48547
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   marka   46290 non-null  object 
 1   naw3    46290 non-null  object 
 2   9at3a   46290 non-null  object 
 3   khochn  46290 non-null  float64
 4   toul    46290 non-null  float64
 5   3ordh   46290 non-null  float64
 6   R       46290 non-null  int64  
 7   G       46290 non-null  int64  
 8   B       46290 non-null  int64  
 9   soum    46290 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 3.9+ MB


In [10]:
data.nunique()

marka         5
naw3          7
9at3a         5
khochn      170
toul        256
3ordh       125
R           139
G            30
B            40
soum      10978
dtype: int64

In [11]:
data.describe()

,khochn,toul,3ordh,R,G,B,soum
count,46290.000000,46290.000000,46290.000000,46290.000000,46290.000000,46290.000000,46290.000000
mean,1.029034,64.062320,57.481488,135.830676,24.366775,27.920696,396.606492
std,0.023476,37.529743,2.231935,26.477142,4.950381,5.645137,399.668502
min,0.716667,16.000000,43.000000,0.000000,0.000000,0.000000,32.600000
25%,1.016667,32.000000,56.000000,112.000000,20.000000,23.000000,95.100000
50%,1.030000,56.800000,57.000000,135.000000,24.000000,28.000000,245.600000
75%,1.041667,83.200000,59.000000,155.000000,28.000000,32.000000,538.400000
max,1.316667,280.800000,95.000000,229.000000,255.000000,255.000000,1882.300000


In [12]:
data = data[data["toul"]<260]
data = data.loc[(data["3ordh"] < 70) & (data["3ordh"] > 51)]
data = data.loc[(data["khochn"] < 1.16) & (data["khochn"] > 0.92)]
data = data.loc[(data["R"] < 240) & (data["R"] > 100)]
data = data.loc[(data["G"] < 50) & (data["G"] > 10)]
data = data.loc[(data["B"] < 53) & (data["B"] > 17)]

In [13]:
X = data.drop('soum', axis = 1)
y = data.soum

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state= 1)
X_train.shape, X_test.shape

((35686, 9), (8922, 9))

In [15]:
preprocessor = TransformationPipeline().preprocess(d=data)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [16]:
dump(preprocessor,"ClothesPreprocessor.joblib")

['ClothesPreprocessor.joblib']

In [16]:
models = {
    'ridge' : Ridge(),
    'xgboost' : XGBRegressor(),
    'catboost' : CatBoostRegressor(verbose=0),
    'lightgbm' : LGBMRegressor(),
}

In [17]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(f'{name} trained')

ridge trained
xgboost trained
catboost trained
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001217 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 658
[LightGBM] [Info] Number of data points in the train set: 35686, number of used features: 23
[LightGBM] [Info] Start training from score 409.089332
lightgbm trained


In [23]:
for name, model1 in models.items():
    predictions = model1.predict(X_test)
    print(name , " r2_score", r2_score(y_test,predictions))
    print(name , " RMSE", np.sqrt(mean_squared_error(y_test,predictions)))
    print(name , " MAE", mean_absolute_error(y_test,predictions))

ridge  r2_score 0.9340987759654636
ridge  RMSE 101.80292049201694
ridge  MAE 69.71960702004243
xgboost  r2_score 0.9800600349212047
xgboost  RMSE 55.99840964370866
xgboost  MAE 29.49223589701547
catboost  r2_score 0.9818340081940078
catboost  RMSE 53.44942696780701
catboost  MAE 28.71945894951966
lightgbm  r2_score 0.981493603663101
lightgbm  RMSE 53.94788535968604
lightgbm  MAE 29.18937245351777


In [20]:
# dump(models["catboost"],"Catboost.joblib")

['Catboost.joblib']

2910.3743347818277